In [5]:
# Package Load and CUDA Setting
import pandas as pd
pd.options.mode.chained_assignment = None

from tqdm.notebook import tqdm

# Torch
import torch 
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchmetrics import F1Score

# Pre-trained ELECTRA
from transformers import (ElectraTokenizerFast,
                          AutoTokenizer,
                          ElectraModel, 
                          ElectraForSequenceClassification,
                          AdamW)
                          
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    map_location=torch.device('cpu')


your device is running on cpu now.


In [6]:
class SentimentClassifier:

  LABELS = ['부정', '긍정', '중립',]
  ID_LABELS = {idx: key for (idx, key) in enumerate(LABELS)}
  DataFrame = pd.DataFrame()

  def __init__(self):
    #self.df=pd.read_csv(PATH,index_col=0) #추가
    self.model = ElectraForSequenceClassification.from_pretrained("kykim/electra-kor-base", problem_type="multi_label_classification", num_labels=3).to(device)
    self.model.load_state_dict(torch.load("model/0727_koelectra_batch32_lr3e-5.pt", map_location=device))
    self.tokenizer = ElectraTokenizerFast.from_pretrained("kykim/electra-kor-base")
    
    # print(self.dataset.describe())
    return print(f"Your device is running on {device} now.")
    
    
  def help(self):
    
    return print("""
    1. 클래스 객체 설명 
    클래스 객체를 생성하면 transformers로부터 electra-kor-base 모델을 불러온 뒤, 저장된 파일로부터 파라미터 값을 불러옵니다. 또한, 이에 맞는 토크나이저를 함께 불러옵니다. 이 모델과 토크나이저는 클래스 객체 내부에 저장됩니다.
    하위폴더 /model에 올바른 이름의 모델이 위치해야 합니다. 

    2. 클래스 내부 함수 설명
      2-1. predict(text) : 문자열 형태로 전달하면 긍정/부정/중립 중 하나의 결과를 제공합니다.
      2-2. labeling(DataFrame) : INPUT DATA는 판다스 데이터프레임 형태로 전달합니다. 데이터프레임의 title 컬럼의 문자열들을 감성분석한 결과를 emotion 컬럼에 담아 제공합니다.
      2-3. get_DataFrame() : 이 객체를 통해 변환된 데이터프레임을 전달합니다.

    """)

  def _get_prediction_input(self, text):
    inputs = self.tokenizer(
        text, 
        return_tensors = 'pt',
        truncation = True,
        #max_length = MAX_LEN,
        padding = 'longest',
        #pad_to_max_length = True,
        add_special_tokens = True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask

  def predict(self, text):
    input_ids, attention_mask = self._get_prediction_input(text)
    y_pred = self.model(input_ids.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))[0]
    _, predicted = torch.max(y_pred, 1)
    return SentimentClassifier.ID_LABELS[predicted.item()]

  def labeling(self, DataFrame):
    self.DataFrame = DataFrame
    self.DataFrame['emotion'] = self.DataFrame['title'].apply(self.predict)
    return self.DataFrame

  def get_DataFrame(self):
    return self.DataFrame